In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import utils
from tensorflow import keras

import numpy as np

# Генератор аугментированных изображений
from keras.preprocessing.image import ImageDataGenerator

# Основные слои
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, BatchNormalization, ZeroPadding2D

# Оптимизатор
from keras.optimizers import Adam

# Матрица ошибок классификатора
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Подключение модуля для загрузки данных из облака
import gdown

# Инструменты для работы с файлами
import os

# Отрисовка графиков
import matplotlib.pyplot as plt


# Рисование графиков в ячейках Colab
%matplotlib inline

import zipfile


In [ ]:
zip_path = '/content/drive/MyDrive/archive.zip'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall('/content/')

In [ ]:
# Запишем путь к данным и создадим генераторы для обучающей и тестовой выборок

data_dir = '/content/train'
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(48, 48),
    batch_size=32,color_mode='grayscale',
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(48, 48),
    batch_size=32, color_mode='grayscale',
    class_mode='categorical',
    subset='validation'
)

test_generator = test_datagen.flow_from_directory(
    '/content/test',
    target_size=(48, 48),
    batch_size=32, color_mode='grayscale',
    class_mode='categorical',
    shuffle=False
)

Found 22968 images belonging to 7 classes.
Found 5741 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [ ]:
class_names = list(train_generator.class_indices.keys())
print("Имена классов:", class_names)

Имена классов: ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']


In [ ]:
model = Sequential()
model.add(ZeroPadding2D((1,1), input_shape=(48, 48, 1)))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(128, (3,3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(128, (3,3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(256, (3,3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(256, (3,3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(256, (3,3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(512, (3,3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(512, (3,3), activation='relu'))

model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))

model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(train_generator, batch_size=32, epochs=30, verbose=1, validation_data=validation_generator)

Epoch 1/30
718/718 [==============================] - 45s 52ms/step - loss: 1.8344 - accuracy: 0.2495 - val_loss: 1.8141 - val_accuracy: 0.2513
Epoch 2/30
718/718 [==============================] - 34s 48ms/step - loss: 1.8143 - accuracy: 0.2512 - val_loss: 1.8109 - val_accuracy: 0.2513
Epoch 3/30
718/718 [==============================] - 38s 53ms/step - loss: 1.8120 - accuracy: 0.2513 - val_loss: 1.8088 - val_accuracy: 0.2513
Epoch 4/30
718/718 [==============================] - 34s 48ms/step - loss: 1.8096 - accuracy: 0.2513 - val_loss: 1.8045 - val_accuracy: 0.2513
Epoch 5/30
718/718 [==============================] - 35s 48ms/step - loss: 1.7987 - accuracy: 0.2513 - val_loss: 1.7940 - val_accuracy: 0.2520
Epoch 6/30
718/718 [==============================] - 35s 48ms/step - loss: 1.7853 - accuracy: 0.2505 - val_loss: 1.7763 - val_accuracy: 0.2547
Epoch 7/30
718/718 [==============================] - 35s 49ms/step - loss: 1.7740 - accuracy: 0.2565 - val_loss: 1.7621 - val_accuracy:

In [ ]:
loss = model.evaluate(test_generator)
print(loss)

225/225 [==============================] - 3s 13ms/step - loss: 1.5484 - accuracy: 0.5276
[1.5484086275100708, 0.5275843143463135]


In [ ]:
model_json = model.to_json()
with open('model.json', 'w') as json_file:
  json_file.write(model_json)
  model.save_weights('model.h5')
print('saved model to disk')


saved model to disk
